# Geracao da Massa de Dados

## Cria Objetos no Mysql

In [ ]:
CREATE TABLE `tb_event` (
  `event_id` bigint DEFAULT NULL,
  `event_value` double DEFAULT NULL,
  `event_dt` timestamp NULL DEFAULT NULL,
  `device_id` bigint DEFAULT NULL,
  `uf_id` VARCHAR(2) DEFAULT NULL,
  `vehicle_type` VARCHAR(20) DEFAULT NULL
) 

CREATE TABLE `tb_device` (
  `device_id` bigint DEFAULT NULL,
  `device_name` varchar(100) DEFAULT NULL,
  `device_type` varchar(100) DEFAULT NULL,
  `device_status` varchar(100) DEFAULT NULL
) 

In [ ]:
insert into tb_device values(100, "Temperature Sensor", "Temperature","on")
insert into tb_device values(200, "Latitude Sensor", "Latitude","on")
insert into tb_device values(300, "Longitude Sensor", "Longitude","on")
insert into tb_device values(400, "Motion Sensor", "motion","on")
insert into tb_device values(500, "Light Sensor", "Light","on")
insert into tb_device values(600, "Light Sensor", "Light","off")
insert into tb_device values(700, "Temperature Sensor", "Temperature","off")
insert into tb_device values(800, "Temperature Sensor", "Temperature","off")

## Carga MySql

In [ ]:
import random
from datetime import datetime

import pymysql

# Create a connection object
dbServerName    = "database-2.chk1ptphcukm.us-east-1.rds.amazonaws.com"
dbUser          = "admin"
dbPassword      = "xxxxxx"
dbName          = "db06"
charSet         = "utf8mb4"
print("Conectando")
conn   = pymysql.connect(host=dbServerName, user=dbUser, password=dbPassword,db=dbName, charset=charSet,connect_timeout=20)
print("Conexao efetuada")


def insert(event_id,event_value,event_dt,device_id,conn,uf_id,vehicle_type):
    cursorObject            = conn.cursor()   
    cursorObject.execute("use db06;")
    sqlCreateTableCommand   = "INSERT INTO tb_event values({},{},STR_TO_DATE(\'{}\' , '%Y-%m-%d %H:%i:%s.%f'),{},\'{}\',\'{}\');".format(event_id,event_value,event_dt,device_id,uf_id,vehicle_type)
    print(sqlCreateTableCommand)
    cursorObject.execute(sqlCreateTableCommand)
    print("Insert efetuado")


veicle_list = ["TRUNC", "CAR", "BUS"]
for veicle in veicle_list:
    uf_list = ["SP", "MG", "RJ","RS", "SC"]
    for uf in uf_list:
        device=100
        print("Efetuando insert para o device: "+ str(device))
        try:
            for i in range(1000):
                event_id=i
                event_value=random.uniform(18.1, 37.9)
                event_dt=datetime.now()
                device_id=device
                uf_id=uf
                vehicle_type=veicle
                insert(event_id,event_value,event_dt,device_id,conn,uf_id,vehicle_type)
                
                print("-Efetuando commit")
            conn.commit()
        except Exception as e:
            print('Erro ao Executar Carga no SQLServer' + str(e))
            conn.rollback()
            conn.close()
            raise e
        
        device=200
        print("Efetuando insert para o device: "+ str(device))
        try:
            for i in range(900):
                event_id=i
                event_value=random.uniform(-233344.1, -234344.1)
                event_dt=datetime.now()
                device_id=device
                uf_id=uf
                vehicle_type=veicle
                insert(event_id,event_value,event_dt,device_id,conn,uf_id,vehicle_type)
        
            print("-Efetuando commit")
            conn.commit()
        except Exception as e:
            print('Erro ao Executar Carga no SQLServer' + str(e))
            conn.rollback()
            raise e
        
        device=300
        print("Efetuando insert para o device: "+ str(device))
        try:
            for i in range(800):
                event_id=i
                event_value=random.uniform(-463951.9, -465951.9)
                event_dt=datetime.now()
                device_id=device
                uf_id=uf
                vehicle_type=veicle
                insert(event_id,event_value,event_dt,device_id,conn,uf_id,vehicle_type)
        
            print("-Efetuando commit")
            conn.commit()
        except Exception as e:
            print('Erro ao Executar Carga no SQLServer' + str(e))
            conn.rollback()
            conn.close()
            raise e
        
        device=400
        print("Efetuando insert para o device: "+ str(device))
        try:
            for i in range(800):
                event_id=i
                event_value=random.uniform(-463951.9, -465951.9)
                event_dt=datetime.now()
                device_id=device
                uf_id=uf
                vehicle_type=veicle
                insert(event_id,event_value,event_dt,device_id,conn,uf_id,vehicle_type)
        
            print("-Efetuando commit")
            conn.commit()
        except Exception as e:
            print('Erro ao Executar Carga no SQLServer' + str(e))
            conn.rollback()
            conn.close()
            raise e

print("FIM DO PROCESSAMENTO")
conn.close()

## Tabelas no athena

In [ ]:
DROP DATABASE `dl-workshop-refined`
CREATE DATABASE `dl-workshop-refined`

drop table `ft_event`
create external table `ft_event` (
event_id int,
event_value double, 
insert_dt timestamp,
sk_dim_uf_origem int,
sk_dim_vehicle_type int,
sk_dim_device int,
sk_dim_tempo int
)
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
LOCATION
  's3://dl-workshop-refined/ft_event/'
  
  
  
drop table `dim_device`
create external table `dim_device` (
sk_dim_device int,
device_id int, 
device_name string,
device_type string,
device_status string,
dat_inclusao_registro timestamp
)
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
LOCATION
  's3://dl-workshop-refined/dim_device/'
  
select * from dim_device
select * from ft_event
  